In [4]:
import logging
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers, launch_es
from haystack.nodes import FARMReader, TransformersReader, BM25Retriever
import requests

In [7]:
# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="sampledata")

In [8]:
doc_dir = "../data/sampleData"
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)
document_store.write_documents(docs)

INFO - haystack.utils.preprocessing -  Converting ..\data\sampleData\0_Toilet_Training_issues_1.txt
INFO - haystack.utils.preprocessing -  Converting ..\data\sampleData\1_Toilet_Training_issues_2.txt
INFO - haystack.utils.preprocessing -  Converting ..\data\sampleData\2_Toilet_Training_issues_3.txt


In [9]:
requests.get('http://localhost:9200/sampledata/_count').json()

{'count': 6,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [10]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack.retriever.dense import DensePassageRetriever

In [11]:
doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='sampledata'
)

In [13]:
retriever = DensePassageRetriever(
    document_store=doc_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  ======

In [14]:
doc_store.update_embeddings(retriever=retriever)

INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 6 docs ...
Updating embeddings: 10000 Docs [00:05, 1939.13 Docs/s]      


In [16]:
retriever.retrieve("How do I toilet train my child as he keeps wetting himself?")

[<Document: {'content': 'Before you start to toilet train, it is important to make sure that your child is \ncomfortable in the bathroom. If your child is not comfortable, make the bathroom \na fun place! You can do this is many a ways, either read a book with your child in \nthe bathroom with them, play their favorite music and have a dance party with \nthem, etc. Also, before starting it is very helpful to make sure that your child is \nconstantly drinking fluids, as that will ensure that there is more times for you and \nyour child to practice using the toilet appropriately. Another helpful thing to use, \nis a chart to show your childâ€™s progress. You can include the date, time, accidents, \nsuccessful uses of the toilet, and successful initiations by your child.  \nOnce your child is comfortable, then itâ€™s time to set up a routine. Toilet training \ntakes time and patience. Set a kitchen timer at regular intervals, anywhere from \n10-20 minutes. When the timer goes off, show yo